In [ ]:
from numpy.lib.function_base import percentile
import scipy.io
import pandas as pd
import csv
import numpy as np
import pandas as pd
import os
from PIL import Image
import subprocess
import cv2
from tqdm import tqdm
import sys
import random
from matplotlib import pyplot as plt
from ipywidgets import interact, Select
import mediapipe as mp
import re
from scipy.spatial.transform import Rotation

In [ ]:
txt_folder = '/MPIIGaze/Evaluation Subset/sample list for eye image/'
folder_name = '/HDD_1/MPIIGaze/Data/Normalized'
original_file_path = '/MPIIGaze/Data/Original/'
save_img_folder = ''
save_csv_name = ''
os.makedirs(save_img_folder, exist_ok=True)



In [ ]:
def read_txt(txt_filename):
    with open(txt_filename, 'r', encoding='UTF-8') as f:
        data_list = f.readlines()
        # print(data_list)

    return data_list

In [ ]:
def show_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()
    
def show_gaze_img(img, gaze):
    img = img[:,:,np.newaxis]
    img = np.concatenate([img,img,img], axis=2)
    img = cv2.resize(img, dsize=None, fx=3, fy=3)
    y = img.shape[0]
    x = img.shape[1]
    x_2 = int(x/2)
    y_2 = int(y/2)
    img = cv2.circle(img, (x_2, y_2), 1, (0,0,255), thickness = 1)
    img = cv2.arrowedLine(img, (x_2, y_2), (x_2 + int(200*gaze[0]), y_2 + int(200*gaze[1])), (255, 0, 0), thickness=1)
    show_img(img)
    
def show_gaze_img_both(img, gaze_R, gaze_L):
    img = img[:,:,np.newaxis]
    img = np.concatenate([img,img,img], axis=2)
    img = cv2.resize(img, dsize=None, fx=3, fy=3)
    y = img.shape[0]
    x = img.shape[1]
    x_R = int(x/4)
    y_R = int(y/2)
    x_L = int(x*3/4)
    y_L = int(y/2)
    img = cv2.circle(img, (x_R, y_R), 1, (0,0,255), thickness = 1)
    img = cv2.circle(img, (x_L, y_L), 1, (0,0,255), thickness = 1)
    img = cv2.arrowedLine(img, (x_R, y_R), (x_R + int(200*gaze_R[0]), y_R + int(200*gaze_R[1])), (255, 0, 0), thickness=1)
    img = cv2.arrowedLine(img, (x_L, y_L), (x_L + int(200*gaze_L[0]), y_L + int(200*gaze_L[1])), (255, 0, 0), thickness=1)
    show_img(img)
    

In [ ]:
'''---------------------Get TEST DATA LIST to exclude from data for evaluation------------'''
txt_list = os.listdir(txt_folder)
txt_list.sort()

img_data_list = []
gaze_data_list = []
test_dataname_list = []
duplicated_data_name_list = []

for i in range(len(txt_list)):
    '''Get person folder name'''
    test_person_folder_name = txt_list[i]
    test_person_folder_name = test_person_folder_name[:-4]
    # print(test_person_folder_name)

    '''Get data name'''
    txt_file_name = os.path.join(txt_folder, txt_list[i])
    # print(txt_file_name)
    data_name_list = read_txt(txt_file_name)
    # print(np.array(data_name_list).shape)
    for j in range(len(data_name_list)):
        if data_name_list[j][-5:-1] == 'left':
            data_name = data_name_list[j][:-6]
            test_dataname = os.path.join(test_person_folder_name, data_name)
            if test_dataname not in test_dataname_list:
                test_dataname_list.append(test_dataname)
            else:
                duplicated_data_name_list.append(test_dataname)

test_dataname_list = sorted(test_dataname_list)
print(len(test_dataname_list))

In [ ]:
'''----------------------Get Image & Gaze data-----------------------'''
person_files = os.listdir(folder_name)
person_files = sorted(person_files, key=lambda s: int(re.search(r'\d+', s).group()))
gaze_list_left = []
test_cnt = 1

for i in tqdm(range(0,len(test_dataname_list)-1)): 
    test_data_name = test_dataname_list[i]
    # print(test_data_name)
    original_img_folder_path = os.path.join(original_file_path, test_data_name)
    original_img = cv2.imread(original_img_folder_path)
    # print('original_img_folder_path', original_img_folder_path)
    mat_file_path = os.path.join(folder_name, test_data_name[:-9])
    mat_file = scipy.io.loadmat(mat_file_path)
    # print(mat_file_path)
    k = int(test_data_name[-8:-4])-1

    '''Extruct Image & Gaze Data'''
    img_data_left = mat_file["data"]["left"][0][0][0][0][1][k]
    gaze_data_left = mat_file["data"]["left"][0][0][0][0][0][k]
    # print('gaze_data_left: ',gaze_data_left)
    # gaze_data_left_output = [test_cnt,gaze_data_left[0],gaze_data_left[1],gaze_data_left[2]]
    head_pose_left = mat_file["data"]["left"][0][0][0][0][2][k]
    rot_vec = [head_pose_left[2], head_pose_left[1], head_pose_left[0]]
    r = Rotation.from_rotvec(rot_vec)
    # r = Rotation.from_euler('xyz',[rot_vec])
    r_inv = r.inv()
    normalized_gaze = r_inv.apply(np.array(gaze_data_left))
    normalized_gaze = np.squeeze(normalized_gaze)
    normalized_gaze_out = [test_cnt, normalized_gaze[0], normalized_gaze[1], normalized_gaze[2]]
    # print('normalized_gaze_out: ',normalized_gaze_out)
    img_name = str(test_cnt) + '.png'
    img_path = os.path.join(save_img_folder, img_name)

    cv2.imwrite(img_path, np.array(img_data_left))
    gaze_list_left.append(normalized_gaze_out)
    test_cnt +=1

'''Save CSV File'''
df_v = pd.DataFrame(gaze_list_left)
df_v.to_csv(save_csv_name,index=False,header = False)
print('save test csv')

    
